In [4]:
import pandas as pd
import os

# 请确保这个路径和您之前代码中的路径一致
TOTAL_BOX_OFFICE_FILE = '../data/电影总票房统计表.csv'

try:
    df_check = pd.read_csv(TOTAL_BOX_OFFICE_FILE)
    print("您文件的【实际列名】是:")
    print(list(df_check.columns))
except FileNotFoundError:
    print(f"错误：依然找不到文件，请确认路径 '{TOTAL_BOX_OFFICE_FILE}' 是否正确。")

您文件的【实际列名】是:
['片名', ' 票房(万元)']


In [10]:
!pip install openpyxl

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ---------------------------------------- 0.0/250.9 kB ? eta -:--:--
     ---- ---------------------------------- 30.7/250.9 kB 1.4 MB/s eta 0:00:01
     --------- ----------------------------- 61.4/250.9 kB 1.1 MB/s eta 0:00:01
     --------------------- ---------------- 143.4/250.9 kB 1.1 MB/s eta 0:00:01
     -------------------------- ----------- 174.1/250.9 kB 1.1 MB/s eta 0:00:01
     -------------------------------------- 250.9/250.9 kB 1.3 MB/s eta 0:00:00


In [6]:
import os

# 1. 打印当前工作目录
current_working_dir = os.getcwd()
print(f"Notebook 当前的工作目录是: {current_working_dir}")

# 2. 测试我们预期的相对路径
data_path_test = '../data'
full_path_test = os.path.abspath(data_path_test)

print(f"\n我们代码中使用的相对路径是: '{data_path_test}'")
print(f"它解析出的绝对路径是: {full_path_test}")

# 3. 检查这个路径是否存在
if os.path.exists(full_path_test) and os.path.isdir(full_path_test):
    print("\n✅ 诊断成功：根据这个路径，data 文件夹被找到了！")
    print("   如果依然报错，问题可能在于后续的文件名（例如 '电影总票房统计表.csv'）或读取权限。")
else:
    print("\n❌ 诊断失败：根据这个路径，data 文件夹【没有】被找到！")
    print("   这说明您Jupyter Notebook的启动目录可能不是我们预期的项目根目录。")

Notebook 当前的工作目录是: D:\App\Pycharm\1117MovieSchedulingSimulation\jupyters

我们代码中使用的相对路径是: '../data'
它解析出的绝对路径是: D:\App\Pycharm\1117MovieSchedulingSimulation\data

✅ 诊断成功：根据这个路径，data 文件夹被找到了！
   如果依然报错，问题可能在于后续的文件名（例如 '电影总票房统计表.csv'）或读取权限。


In [5]:
import pandas as pd
import os

# --- 路径配置 ---
DATA_DIR = '..\data' 
TOTAL_BOX_OFFICE_FILE = os.path.join(DATA_DIR, '电影总票房统计表.csv')
DAILY_DATA_FOLDER = os.path.join(DATA_DIR, '2019-2024电影每日票房信息')


# ========================= [ 关键修改处 1：修正阈值单位 ] =========================
# 将阈值从小数 (0-1) 调整为与数据一致的百分比数字 (0-100)
# =================================================================================
# “黑马”定义：总票房很高，但首日排片很低
DARK_HORSE_GROSS_THRESHOLD = 50000      # 总票房高于 50,000 万元 (即5亿)
DARK_HORSE_ALLOCATION_THRESHOLD = 20    # 首日场次占比低于 20%

# “烂片”定义：总票房很低，但首日排片很高
FLOP_GROSS_THRESHOLD = 15000            # 总票房低于 15,000 万元 (即1.5亿)
FLOP_ALLOCATION_THRESHOLD = 20        # 首日场次占比高于 18%


def find_special_movies():
    """
    从数据中筛选出符合“黑马”和“烂片”定义的电影。
    """
    dark_horse_candidates = []
    flop_candidates = []

    print(f"正在加载总票房数据: {TOTAL_BOX_OFFICE_FILE}")
    try:
        total_gross_df = pd.read_csv(TOTAL_BOX_OFFICE_FILE)
        if '总票房（亿）' in total_gross_df.columns:
            total_gross_df.rename(columns={'总票房（亿）': '总票房(亿)'}, inplace=True)
    except FileNotFoundError:
        print(f"错误：找不到总票房文件。")
        return [], []

    print(f"共加载 {len(total_gross_df)} 部电影的总票房数据。")
    print("\n--- 开始寻找“黑马”和“烂片”候选者 ---")

    for index, row in total_gross_df.iterrows():
        try:
            movie_name = row['片名']
            total_gross_in_wan = row[' 票房(万元)']

            daily_file_path = os.path.join(DAILY_DATA_FOLDER, f'{movie_name}.xlsx')
            
            if os.path.exists(daily_file_path):
                daily_df = pd.read_excel(daily_file_path)
                if daily_df.empty or '场次占比' not in daily_df.columns:
                    continue

                # 这里的 first_day_allocation 是一个 0-100 的数字, 例如 15.0
                first_day_allocation = daily_df.iloc[0]['场次占比']

                # 判断是否为“黑马” (现在是正确的百分比数字比较)
                if total_gross_in_wan > DARK_HORSE_GROSS_THRESHOLD and first_day_allocation < DARK_HORSE_ALLOCATION_THRESHOLD:
                    candidate = {
                        'name': movie_name,
                        'total_gross_Wan': total_gross_in_wan,
                        # ========================= [ 关键修改处 2：移除多余的 * 100 ] =========================
                        'first_day_allocation_pct': first_day_allocation 
                    }
                    dark_horse_candidates.append(candidate)

                # 判断是否为“烂片” (现在是正确的百分比数字比较)
                if total_gross_in_wan < FLOP_GROSS_THRESHOLD and first_day_allocation > FLOP_ALLOCATION_THRESHOLD:
                    candidate = {
                        'name': movie_name,
                        'total_gross_Wan': total_gross_in_wan,
                        # ========================= [ 关键修改处 2：移除多余的 * 100 ] =========================
                        'first_day_allocation_pct': first_day_allocation
                    }
                    flop_candidates.append(candidate)
        
        except (KeyError, AttributeError):
            continue
    
    return dark_horse_candidates, flop_candidates

# --- 运行并输出结果 ---
dark_horses, flops = find_special_movies()
    
print("\n" + "="*60)
print("“黑马”电影候选名单总结:")
if not dark_horses:
    print("未找到符合条件的“黑马影片”候选者。请尝试进一步放宽阈值。")
else:
    for movie in sorted(dark_horses, key=lambda x: x['total_gross_Wan'], reverse=True):
        gross_in_billion = movie['total_gross_Wan'] / 10000
        print(f" - 《{movie['name']}》 (最终票房: {gross_in_billion:.2f}亿, 首日排片: {movie['first_day_allocation_pct']:.2f}%)")
    
print("\n" + "="*60)
print("“烂片”电影候选名单总结:")
if not flops:
    print("未找到符合条件的“高开低走影片”候选者。请尝试进一步放宽阈值。")
else:
    for movie in sorted(flops, key=lambda x: x['first_day_allocation_pct'], reverse=True):
        gross_in_billion = movie['total_gross_Wan'] / 10000
        print(f" - 《{movie['name']}》 (最终票房: {gross_in_billion:.2f}亿, 首日排片: {movie['first_day_allocation_pct']:.2f}%)")
print("="*60)

正在加载总票房数据: ..\data\电影总票房统计表.csv
共加载 2385 部电影的总票房数据。

--- 开始寻找“黑马”和“烂片”候选者 ---

“黑马”电影候选名单总结:
 - 《流浪地球》 (最终票房: 46.62亿, 首日排片: 11.49%)
 - 《这个杀手不太冷静》 (最终票房: 26.16亿, 首日排片: 14.72%)
 - 《封神第一部：朝歌风云》 (最终票房: 25.70亿, 首日排片: 17.81%)
 - 《第二十条》 (最终票房: 24.49亿, 首日排片: 19.49%)
 - 《熊出没·逆转时空》 (最终票房: 19.80亿, 首日排片: 11.27%)
 - 《长安三万里》 (最终票房: 17.88亿, 首日排片: 19.40%)
 - 《熊出没·伴我“熊芯”》 (最终票房: 14.78亿, 首日排片: 8.65%)
 - 《我和我的父辈》 (最终票房: 14.50亿, 首日排片: 16.10%)
 - 《坚如磐石》 (最终票房: 13.48亿, 首日排片: 12.00%)
 - 《人生路不熟》 (最终票房: 11.81亿, 首日排片: 10.80%)
 - 《悬崖之上》 (最终票房: 11.72亿, 首日排片: 16.93%)
 - 《刺杀小说家》 (最终票房: 10.35亿, 首日排片: 12.98%)
 - 《前任4：英年早婚》 (最终票房: 10.10亿, 首日排片: 16.44%)
 - 《比悲伤更悲伤的故事》 (最终票房: 9.57亿, 首日排片: 11.58%)
 - 《熊出没·重返地球》 (最终票房: 9.54亿, 首日排片: 5.81%)
 - 《无名》 (最终票房: 9.29亿, 首日排片: 12.95%)
 - 《扬名立万》 (最终票房: 9.23亿, 首日排片: 16.02%)
 - 《深海》 (最终票房: 9.14亿, 首日排片: 8.94%)
 - 《我的姐姐》 (最终票房: 8.60亿, 首日排片: 16.58%)
 - 《志愿军：雄兵出击》 (最终票房: 8.27亿, 首日排片: 14.58%)
 - 《你想活出怎样的人生》 (最终票房: 7.91亿, 首日排片: 19.93%)
 - 《异形：夺命舰》 (最终票房: 7.85亿, 首日排片: 14.04%)
 - 《末路狂花钱》 (最终票房

In [1]:
import pandas as pd
import os

# 要查找的电影名
movie_name = '1917' 

# 构建文件路径
daily_file_path = f"../data/2019-2024电影每日票房信息/{movie_name}.xlsx"

if os.path.exists(daily_file_path):
    daily_df = pd.read_excel(daily_file_path)
    
    # 假设日期信息在'日期/上映天数'列，并用'|'分隔
    # 这部分逻辑与您的 simsche.py 环境代码一致
    release_date_str = daily_df.iloc[0]['日期/上映天数'].split('|')[0]
    
    print(f"根据您的数据文件，《{movie_name}》的上映日期是: {release_date_str}")
else:
    print(f"找不到《{movie_name}》的每日数据文件。")

根据您的数据文件，《1917》的上映日期是: 2020-08-07


In [16]:
import pandas as pd
import os

# 要查找的电影名
movie_name = '这个杀手不太冷静' 

# 构建文件路径
daily_file_path = f"../data/2019-2024电影每日票房信息/{movie_name}.xlsx"

if os.path.exists(daily_file_path):
    daily_df = pd.read_excel(daily_file_path)
    
    # 假设日期信息在'日期/上映天数'列，并用'|'分隔
    # 这部分逻辑与您的 simsche.py 环境代码一致
    release_date_str = daily_df.iloc[0]['日期/上映天数'].split('|')[0]
    
    print(f"根据您的数据文件，《{movie_name}》的上映日期是: {release_date_str}")
else:
    print(f"找不到《{movie_name}》的每日数据文件。")

根据您的数据文件，《这个杀手不太冷静》的上映日期是: 2022-02-01


In [17]:
import pandas as pd
import os

# 要查找的电影名
movie_name = '熊出没·伴我“熊芯”' 

# 构建文件路径
daily_file_path = f"../data/2019-2024电影每日票房信息/{movie_name}.xlsx"

if os.path.exists(daily_file_path):
    daily_df = pd.read_excel(daily_file_path)
    
    # 假设日期信息在'日期/上映天数'列，并用'|'分隔
    # 这部分逻辑与您的 simsche.py 环境代码一致
    release_date_str = daily_df.iloc[0]['日期/上映天数'].split('|')[0]
    
    print(f"根据您的数据文件，《{movie_name}》的上映日期是: {release_date_str}")
else:
    print(f"找不到《{movie_name}》的每日数据文件。")

根据您的数据文件，《熊出没·伴我“熊芯”》的上映日期是: 2023-01-22


In [20]:
import pandas as pd
import os

# 要查找的电影名
movie_name = '上海堡垒' 

# 构建文件路径
daily_file_path = f"../data/2019-2024电影每日票房信息/{movie_name}.xlsx"

if os.path.exists(daily_file_path):
    daily_df = pd.read_excel(daily_file_path)
    
    # 假设日期信息在'日期/上映天数'列，并用'|'分隔
    # 这部分逻辑与您的 simsche.py 环境代码一致
    release_date_str = daily_df.iloc[0]['日期/上映天数'].split('|')[0]
    
    print(f"根据您的数据文件，《{movie_name}》的上映日期是: {release_date_str}")
else:
    print(f"找不到《{movie_name}》的每日数据文件。")

根据您的数据文件，《上海堡垒》的上映日期是: 2019-08-09


In [21]:
import pandas as pd
import os

# 要查找的电影名
movie_name = '雷霆沙赞！众神之怒' 

# 构建文件路径
daily_file_path = f"../data/2019-2024电影每日票房信息/{movie_name}.xlsx"

if os.path.exists(daily_file_path):
    daily_df = pd.read_excel(daily_file_path)
    
    # 假设日期信息在'日期/上映天数'列，并用'|'分隔
    # 这部分逻辑与您的 simsche.py 环境代码一致
    release_date_str = daily_df.iloc[0]['日期/上映天数'].split('|')[0]
    
    print(f"根据您的数据文件，《{movie_name}》的上映日期是: {release_date_str}")
else:
    print(f"找不到《{movie_name}》的每日数据文件。")

根据您的数据文件，《雷霆沙赞！众神之怒》的上映日期是: 2023-03-17


In [4]:
import pandas as pd
import os

# 要查找的电影名
movie_name = '跳舞吧！大象' 

# 构建文件路径
daily_file_path = f"../data/2019-2024电影每日票房信息/{movie_name}.xlsx"

if os.path.exists(daily_file_path):
    daily_df = pd.read_excel(daily_file_path)
    
    # 假设日期信息在'日期/上映天数'列，并用'|'分隔
    # 这部分逻辑与您的 simsche.py 环境代码一致
    release_date_str = daily_df.iloc[0]['日期/上映天数'].split('|')[0]
    
    print(f"根据您的数据文件，《{movie_name}》的上映日期是: {release_date_str}")
else:
    print(f"找不到《{movie_name}》的每日数据文件。")

根据您的数据文件，《跳舞吧！大象》的上映日期是: 2019-07-26


In [20]:
import pandas as pd
import os
import sys

# --- [ 步骤 0: 环境设置和路径检查 ] ---
# (确保我们可以访问您的数据文件)
try:
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
    if project_root not in sys.path:
        sys.path.append(project_root)
    
    # 路径配置
    DATA_DIR = os.path.join(project_root, 'data') 
    CALENDAR_FILE_PATH = os.path.join(DATA_DIR, 'movie_calendar.csv')
    
    # 检查文件是否存在
    if not os.path.exists(CALENDAR_FILE_PATH):
        raise FileNotFoundError(f"在路径 '{CALENDAR_FILE_PATH}' 未找到 movie_calendar.csv 文件")

    print(f"✅ 成功定位到文件: {CALENDAR_FILE_PATH}")

except Exception as e:
    print(f"❌ 路径设置或文件检查出错: {e}")


# --- [ 步骤 1: 诊断程序 ] ---
def diagnose_query_function(calendar_path, movie_name, target_date_str):
    """
    一个专门用于诊断 query_valid_movie_name_in_date 函数问题的脚本。
    """
    print("\n" + "="*50)
    print(f"开始诊断电影: 《{movie_name}》")
    print(f"目标查询日期: {target_date_str}")
    print("="*50)

    try:
        # 1. 加载数据
        movie_calendar = pd.read_csv(calendar_path)
        print("\n[1. 数据加载]")
        print("movie_calendar.csv 加载成功。")

        # 2. 查找电影条目
        movie_entry = movie_calendar[movie_calendar['电影名称'] == movie_name]
        if movie_entry.empty:
            print(f"❌ 诊断失败：在文件中找不到名为《{movie_name}》的电影。请检查电影名称是否完全匹配。")
            return

        print("\n[2. 电影条目]")
        print("成功找到电影条目：")
        print(movie_entry.to_string())

        # 3. 提取并检查数据类型
        release_date_from_csv = movie_entry['上映日期'].iloc[0]
        end_date_from_csv = movie_entry['下线日期'].iloc[0]
        
        print("\n[3. 数据类型检查]")
        print(f"从文件中读取的 '上映日期' 原始值: '{release_date_from_csv}' (类型: {type(release_date_from_csv)})")
        print(f"从文件中读取的 '下线日期' 原始值: '{end_date_from_csv}' (类型: {type(end_date_from_csv)})")

        # 4. 模拟日期转换
        current_date_ts = pd.to_datetime(target_date_str)
        release_date_ts = pd.to_datetime(release_date_from_csv)
        end_date_ts = pd.to_datetime(end_date_from_csv)
        
        print("\n[4. 日期转换后]")
        print(f"目标查询日期 (转换后): {current_date_ts}")
        print(f"上映日期 (转换后): {release_date_ts}")
        print(f"下线日期 (转换后): {end_date_ts}")
        
        # 5. 模拟核心判断逻辑 (使用修正后的 <= 和 >=)
        is_after_release = release_date_ts <= current_date_ts
        is_before_end = end_date_ts >= current_date_ts
        final_decision = is_after_release and is_before_end

        print("\n[5. 核心逻辑判断]")
        print(f"判断1: ('上映日期' <= '查询日期') => ({release_date_ts} <= {current_date_ts}) => 结果: {is_after_release}")
        print(f"判断2: ('下线日期' >= '查询日期') => ({end_date_ts} >= {current_date_ts}) => 结果: {is_before_end}")
        print("-" * 20)
        print(f"最终综合判断结果 ('判断1' AND '判断2'): {final_decision}")
        
        print("\n" + "="*50)
        if not final_decision:
            print(">>> 结论：问题找到了！<<<")
            if not is_after_release:
                print("失败原因在于【判断1】。这通常意味着文件中的'上映日期'由于某种原因（例如包含了具体时间）比查询日期的零点要晚。")
            elif not is_before_end:
                 print("失败原因在于【判断2】。这通常意味着文件中的'下线日期'比查询日期要早。")
        else:
            print(">>> 结论：奇怪，根据这份诊断，判断应该是成功的。这可能意味着其他地方还有问题。")
        print("="*50)

    except Exception as e:
        print(f"\n诊断过程中发生错误: {e}")

# --- [ 步骤 2: 运行诊断 ] ---
if 'CALENDAR_FILE_PATH' in locals() and os.path.exists(CALENDAR_FILE_PATH):
    diagnose_query_function(
        calendar_path=CALENDAR_FILE_PATH, 
        movie_name='1917', 
        target_date_str='2019-07-26'
    )

❌ 路径设置或文件检查出错: 在路径 'D:\App\Pycharm\data\movie_calendar.csv' 未找到 movie_calendar.csv 文件


In [13]:
import os
import pandas as pd
from envs.simsche import SimpleScheduler
from tools import Logger

# 初始化一个日志器（关闭打印）
logger = Logger(logdir='logs', is_print=False)

# 创建环境实例
env = SimpleScheduler(logger=logger)

# 固定起始日期，确保包含 “跳舞吧！大象”
state = env.reset(fix_start_date='2019-07-26', ensure_movie='跳舞吧！大象')

# 打印环境当前识别的电影列表
print("\n🎬 环境识别到的 next_day_movie_name 列表:")
for i, name in enumerate(env.next_day_movie_name):
    print(f"  [{i}] {name}")

# 检查是否能找到目标电影
target = '跳舞吧！大象'
if target in env.next_day_movie_name:
    print(f"\n✅ 成功找到电影: {target}, 索引 = {env.next_day_movie_name.index(target)}")
else:
    print(f"\n❌ 未找到电影: {target}")
    print("提示：以下是所有电影名（用于排查字符差异）:")
    for k in env.next_day_movie_name:
        print(repr(k))



NameError: name '__file__' is not defined

In [16]:
from envs.simsche import SimpleScheduler

# --- 定义一个简单的 logger 替身 ---
class DummyLogger:
    def log(self, *args, **kwargs):
        pass  # 什么也不做

dummy_logger = DummyLogger()

# --- 初始化环境 ---
env = SimpleScheduler(logger=dummy_logger)
print("🎬 环境初始化成功！")

# --- 检查加载的数据 ---
print("电影数量:", len(env.movie_calendar))
print(env.movie_calendar.head())


🎬 环境初始化成功！
电影数量: 2385
      电影名称       上映日期       下线日期
0     四个春天 2019-01-04 2019-03-04
1  奎迪：英雄再起 2019-01-04 2019-03-04
2      大黄蜂 2019-01-04 2019-03-04
3     功夫营救 2019-01-04 2019-02-03
4     好好回家 2019-01-05 2019-01-31
